In [1]:
import psycopg2 as pg
import pandas as pd

In [2]:
try:
	connection = pg.connect(
		user = "edd",
		password = "PlatoN03",
		host = "localhost",
		port = "5432",
		database = "edd_db"
	)

	cursor = connection.cursor()
	cursor.execute("SELECT version();")
	record = cursor.fetchone()
	print("Te conectaste - ", record, "\n")

	# Mostar usuario
	cursor.execute("SELECT current_user;")
	record = cursor.fetchone()
	print("Usuario actual - ", record, "\n")
except (Exception, pg.Error) as error:
	print("Error while connecting to PostgreSQL", error)


Te conectaste -  ('PostgreSQL 17.0 (Debian 17.0-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',) 

Usuario actual -  ('edd',) 



In [3]:
def execute_query(connection, query):
	try:
		cursor = connection.cursor()
		cursor.execute(query)
		connection.commit()
		print("Query se ejecuto correctamente.")
	except (Exception, pg.Error) as error:
		print("Error en la ejecucion de la query: ", error)

In [4]:

# Leer el archivo CSV
csv_path = '../data/spotify_songs.csv'
df = pd.read_csv(csv_path)

if connection is not None:
    # Crear esquema db2
    create_schema_query = "CREATE SCHEMA IF NOT EXISTS db2;"
    execute_query(connection, create_schema_query)
    print("Esquema: db2 creado.\n")

    create_table_query = """
    CREATE TABLE IF NOT EXISTS db2.spotify_songs (
            id SERIAL PRIMARY KEY,
            track_name VARCHAR,
            track_artist VARCHAR,
            lyrics TEXT
    );
    """
    cursor.execute(create_table_query)
    print("Tabla: spotify_songs en el esquema db2 creada.\n")

    for index, row in df.iterrows():
        insert_query = """
        INSERT INTO db2.spotify_songs (
                track_name, track_artist, lyrics
        ) VALUES (%s, %s, %s)
        """
        cursor.execute(insert_query, (row['track_name'], row['track_artist'], row['lyrics']))
    
    connection.commit()
    print("Datos insertados en la tabla spotify_songs.\n")

else:
    print("Error en la conexión con la base de datos.")

Query se ejecuto correctamente.
Esquema: db2 creado.

Tabla: spotify_songs en el esquema db2 creada.

Datos insertados en la tabla spotify_songs.



In [5]:
# Crear indice GIN
add_colum_query = """ALTER TABLE db2.spotify_songs ADD COLUMN tsv tsvector;"""
create_indx_query = """CREATE INDEX idx_fts ON db2.spotify_songs USING GIN(tsv);"""
up_query = "UPDATE db2.spotify_songs SET tsv = to_tsvector('spanish', track_name || ' ' || track_artist || ' ' || lyrics);"
trigger_query = """
CREATE TRIGGER tsvectorupdate BEFORE INSERT OR UPDATE ON db2.spotify_songs
FOR EACH ROW EXECUTE FUNCTION tsvector_update_trigger(tsv, 'pg_catalog.spanish', track_name, track_artist, lyrics);
"""

execute_query(connection, add_colum_query)
execute_query(connection, create_indx_query)
execute_query(connection, up_query)
execute_query(connection, trigger_query)


Query se ejecuto correctamente.
Query se ejecuto correctamente.
Query se ejecuto correctamente.
Query se ejecuto correctamente.


In [6]:
query = """
SELECT * FROM db2.spotify_songs WHERE tsv @@ to_tsquery('spanish', 'Diggah & richtig & fies & Sub-Sub-Subwoofer & Bass');
"""

cursor.execute(query)
record = cursor.fetchall()

for row in record:
		print(row)



(199, 'Ich Binnezz', 'Bonez MC', "Diggah richtig fies Sub-Sub-Subwoofer Bass pumpt richtig tief Ficker für die Enemies Flagge aus'm Autofenster find ich mies Boomshalaklak wer macht Umsatz bei Nacht Yo, Baga mit der Tasche voller funkelnder Buds Kapuziner, Mandrill unfassbar krass Herz pumpt 808 wie ein Untergangsbass Dass er diese Schiene ernsthaft fährt Man ich glaube die Musik hat sein Gehirn zerstört Guck in Spiegel sag für mich ist dieser Kerl was wert Denn der Typ strahlt aus wie ein Kernkraftwerk Menschen werden nicht durch ihre Gegend zugekifft Du hast keine Meinung solange du nicht weißt wer du bist Mieses Genie wir waren nie so wie die Hamburger Rapper reden von Melancholie (tss) Capa Zeen MC Mehrfach gespaltet so wie'n Schizophrenie Doch ich bleib Untergrund mit paar hundert Jungs Wenn wir kommen wird es schon mal bunt unter uns Niemand hält den Hunger stumm Denn wir lungern und betteln mit unseren Nummern rum Restart 187ers zweifeln niemals an der Ziehkraft meiner Features 

In [7]:



def mostrar_datos_tabla(cursor, limit=None):
	try:
		query = "SELECT * FROM db2.spotify_songs"
		if limit:
			query += f" LIMIT {limit}"
		cursor.execute(query)
		records = cursor.fetchall()
		df = pd.DataFrame(records, columns=['id', 'track_name', 'track_artist', 'lyrics', 'tsv'])
		return df
	except (Exception, pg.Error) as error:
		print("Error al obtener los datos de la tabla: ", error)

# Ejemplo de uso
df_songs = mostrar_datos_tabla(cursor, limit=10)
df_songs

,id,track_name,track_artist,lyrics,tsv
0,7,Satisfy You,Diddy,All I want is somebody who's gonna love me for...,"'ador':675 'ain':31,136,209,273,278,409,562 'a..."
1,14,Changes,2Pac,"I see no changes, wake up in the morning and I...",'2pac':2 'about':52 'actin':288 'after':726 'a...
2,112,Payback - Original Mix,Sandro Silva,NaN,'mix':3 'nan':6 'original':2 'payback':1 'sand...
3,122,How Ya Like Me Now - Longer Version,Kool Moe Dee,"NA I throw my tape on then, I watch ya Three s...","'about':347,617 'again':376 'ain':91 'all':185..."
4,125,No Hands (feat. Roscoe Dash & Wale),Waka Flocka Flame,"Listen to this track, bitch! Girl, the way you...","'16':464,1278 '50':174,988 '70':171,985 'after..."
5,3737,Definition of House Music,The Bloody Beetroots,Lyrics for this song have yet to be released. ...,'back':19 'be':15 'been':24 'beetroots':7 'blo...
6,199,Ich Binnezz,Bonez MC,Diggah richtig fies Sub-Sub-Subwoofer Bass pum...,"'187ers':159 '808':47 'aber':184 'alles':217,4..."
7,200,Conte a Todos,Sango,NaN,'cont':1 'nan':5 'sang':4
8,201,Lucifer,JAY-Z,"NA Kanyeezy you did it again, you a genius nig...","'abov':303 'after':175 'again':10,583 'ain':10..."
9,203,Middle Of The Night,MJG,"Ahh yeah mic check 1, 2 Eightball and MJG spac...","'1':10 '2':11 'abc':893 'about':57,123 'affa':..."
